# A Simple Workflow implemented in Langgraph

- Langgraph is a sleek workflow used to build agents in a graph like paradigm. 

 Importing all necessary modules.

In [1]:
from typing import Annotated
from typing import Any
from typing_extensions import TypedDict
import os, json, logging
from neo4j import GraphDatabase

#Langgraph and Langchain dependencies. 
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
#from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables.graph_mermaid import draw_mermaid_png

#Manage Environment Variables. 
from dotenv import load_dotenv

load_dotenv()
auradb_conn=os.environ["AURADB_CONN"]
auradb_username = os.environ["AURADB_USERNAME"]
auradb_password = os.environ["AURADB_PASSWORD"]

/Users/raajassode/dev/genai/biograph/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Generate chat response using local LLMs

- We've installed Ollama locally to get access to cutting edge instruct models. 
- The ollama interface allows us to generate responses by loading these large language models locally.
- In this case we'll check out what kind of prompts we'll be giving models as inputs to generate outputs we can utilize. 

In [2]:
passage = """
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also preys on the Sloth (Bradypus variegatus), which has a mutualistic relationship with several species of algae, such as Trentepohlia spp., that grow on its fur, providing camouflage. Additionally, the Sloth shares a symbiotic relationship with the Moth (Cryptoses choloepi), which uses the sloth’s fur for habitat.
From an evolutionary standpoint, the Capybara and the Jaguar belong to different orders: Rodentia and Carnivora, respectively, reflecting deep phylogenetic divergence. The Sloth is taxonomically grouped within the order Pilosa, distinct from the other mammals mentioned.
Together, these species form a complex web of predation, competition, symbiosis, and shared evolutionary history, making the tropical rainforest a dynamic and interconnected ecosystem.
"""
 

## Defining the State Variable 

- An extension of TypedDict, which encapsulates the state of the Graph, which is essentially a list of all the messages, with a reducer function which is called everytime a new message is added.
- All nodes rely on the state to function. Each node takes an input and changes teh state of the applicaiton as the graph progresses. 

In [3]:
class State(TypedDict):
    text: str  # Original input
    json: dict  # LLM output
    messages: Annotated[list, add_messages]


## Defining the Nodes

- Nodes are defined in Langgraph as Units of work. 
- The LLM can invoke these tools as per it's understanding (It reads the docstring) and calls tools based on teh flow of the application. 

In [4]:
def push_graph(state:State) -> Any:

    """ Push the json object with nodes and relationsips between entities to an AuraDB Graph database instance

    Args:
         graph : json object/ Python dictionary that has the graph data stored. with nodes and relationships.

    Returns:
          str : Indicates whether graph push was successful

    """

    AURA_CONNECTION_URI = auradb_conn
    AURA_USERNAME = "neo4j"
    AURA_PASSWORD = auradb_password

    # Driver instantiation
    driver = GraphDatabase.driver(
        AURA_CONNECTION_URI,
        auth=(AURA_USERNAME, AURA_PASSWORD)
    )

    import uuid

    input_data = state["json"] # Your provided JSON here
    
    # Transform nodes
    nodes = {item["node"]: {"id": str(uuid.uuid4())} for item in input_data}

    # Transform relationships
    relationships = []
    for item in input_data:
        source_node = item["node"]
        for rel in item["relationships"]:
            relationships.append({
                "source": source_node,
                "target": rel.get("target", rel.get("source")),
                "type": rel["type"].upper().replace(" ", "_")
            })


    def create_graph(tx):
        # Create nodes with unique IDs
        for name, props in nodes.items():
            nodes_result = tx.run("""
                MERGE (n:Species {name: $name})
                SET n.id = $id
            """, name=name, id=props["id"])

        # Create relationships
        for rel in relationships:
            relationships_result = tx.run("""
                MATCH (a:Species {name: $source}), (b:Species {name: $target})
                MERGE (a)-[r:RELATES_TO {type: $type}]->(b)
            """, source=rel["source"], target=rel["target"], type=rel["type"])
    
        return {"nodes": nodes_result.consume(), "relationships": relationships_result.consume()}

    
    try:
        with driver.session() as session:
            summary = session.execute_write(create_graph)
        
            
            return{"messages":{"role":"assistant","content":f"Graph push successful with {summary['nodes'].counters} nodes and {summary['relationships'].counters}"}}
        
    except Exception as e:
        return{"messages":{"role":"assistant","content":f"Graph push failed with error: {str(e)}"
            }}
    
    
    


In [5]:

llm = ChatOllama(model="mistral:instruct")

In [11]:
def process_passage(state:State):
    prompt = PromptTemplate.from_template("""For the following passage, give me ONLY a serialized json object that 
    describes the relationship between the species, compatible with a graph data structure. 
    For example a graph relationship: Lion --> (predator) --> Antelope should output a serialized json string \n

    the passage starts from the next line \n {text} \n


    *** Requirements ***

        - Only return the json object. Nothing else.
        - each node relationship should have only one source or target. Not a list. 
        - Add any node and relationship properties (ecosystem, etc.) based on the passage

    *** Example output for a relationship ***

    {{
       "node": "Jaguar",
       "relationships": [
         {{
           "type": "predator",
           "target": "Capybara"
         }}
       ]
     }}

    """)
    
    formatted_prompt = prompt.format(text=state["text"])
    response = llm.invoke(formatted_prompt)
    parser = JsonOutputParser()
    payload = {"prompt":response.content,"json":parser.parse(response.content)}
    print(parser.parse(response.content))
    return {
        "prompt": response.content,
        "json": parser.parse(response.content),
        "messages": {"role":"assistant","content":f"Graph created Successfully: \n {payload}"}
    }
    

In [12]:
graph_builder = StateGraph(State)

# Nodes
graph_builder.add_node("process_passage", process_passage)
graph_builder.add_node("push_graph", push_graph)

# Edges
graph_builder.add_edge("process_passage", "push_graph")

# Entry/Exit
graph_builder.set_entry_point("process_passage")
graph_builder.set_finish_point("push_graph")  # Proper termination

graph = graph_builder.compile()

In [13]:
from IPython.display import Image, display

mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	process_passage(process_passage)
	push_graph(push_graph)
	__end__([<p>__end__</p>]):::last
	__start__ --> process_passage;
	process_passage --> push_graph;
	push_graph --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [14]:
import mermaid as md 
from mermaid.graph import Graph 

render = md.Mermaid(mermaid_code)

render

In [15]:
# Initialize with input passage
initial_state = {"text": passage, "messages": []}

# Run the graph
result = graph.invoke(initial_state)

for m in result["messages"]:
    print(m + "\n")

[{'node': 'Jaguar', 'relationships': [{'type': 'predator', 'target': 'Capybara'}, {'type': 'predator', 'target': 'Green Anaconda'}], 'ecosystem': 'tropical rainforest'}, {'node': 'Capybara', 'relationships': [], 'ecosystem': 'tropical rainforest'}, {'node': 'Green Anaconda', 'relationships': [{'type': 'competitor', 'target': 'Harpy Eagle'}], 'ecosystem': 'tropical rainforest'}, {'node': 'Harpy Eagle', 'relationships': [{'type': 'predator', 'target': 'Howler Monkey'}, {'type': 'predator', 'target': 'Sloth'}], 'ecosystem': 'tropical rainforest'}, {'node': 'Howler Monkey', 'relationships': [{'type': 'taxonomically_related', 'target': 'Spider Monkey'}], 'ecosystem': 'tropical rainforest'}, {'node': 'Spider Monkey', 'relationships': [{'type': 'taxonomically_related', 'target': 'Howler Monkey'}], 'ecosystem': 'tropical rainforest'}, {'node': 'Sloth', 'relationships': [{'type': 'symbiont', 'target': 'Trentepohlia spp.'}, {'type': 'habitat_provider', 'target': 'Moth'}], 'ecosystem': 'tropical 